In [1]:
import math
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
import random as rnd
import pandas as pd
import matplotlib.pyplot as plt

Parameters of grid:

In [2]:
JOBS = 1                              # Numbers of jobs
CRITERION = 1e-3                      # Convergence criterion
MAXITERATIONS = 100                   # Maximal number of iterations
PERIODS = 20                          # Number of periods for simulation
AGENTS = 7000                         # Number of agents

AUGMA = 22000                         # Borrowing limit
NBA = 5                              # Number of assets points in the grid
AMIN = 10000                           # Minimal value of assets
AMAX = 500000                         # Maximal value of assets

AUGMH = 4320                          # Maximum hours worked per year
NBH = 50                              # Number of hours worked points in the grid
HMIN = 7000                           # Minimal value of hours worked
HMAX = 85000                          # Maximal value of hours worked

# AMIN = 0.2626
# AMAX = 4.9889

Earnings function parameters (All):

In [3]:
WAGECONSTANT = 2.52                # Utility of being employed
ALPHA = 0.28                       # Performance pay premium
# Earnings premium
ZETA = [0, -.12, -.36, -.21, -.28, -.36, -.16, -.46, -.62] 
# Learning by doing
GAMMA = [0.76, 0.25, -0.02] 
XI = -0.07                         # Depreciation of human capital

Moments of model (All):

In [4]:
PSI = 0.41                         # Labor supply elasticity
CHI = 0.36                         # Disutility of labor supply

Parameters of model:

In [5]:
BETA = 0.98                           # Discount rate
R = 0.04                              # Fixed interest rate
IOTA = 0.5                            # Intertemporal elasticity of substitution
TAU = 0.29                            # Marginal tax rate

ZSHOCKS = 3                           # Number of discrete earnings shocks
z_shock_range = np.array([-1, 0, 1])/20  # Discrete earnings shocks range
pp_range = [0, 1]                     # List of possible PP states

# Probability of having PP given the job choice
PPDDISTRIBUTION = [0.039017, 0.065532, 0.065887, 0.096139, 0.088425, 0.298176, 
            0.106907, 0.134264, 0.105652]
            
# Transition matrix of discrete earnings shocks
PIMATRIX = np.array([7/15, 8/15, 0, 0.4, 0.2, 0.4, 0, 8/15, 7/15])
PIMATRIX = PIMATRIX.reshape(ZSHOCKS, ZSHOCKS)

# Probability of having discrete earnings shock
ZDISTRIBUTION = [0.3, 0.4, 0.3]

o_range = [False, True]               # List of possible o states

In [6]:
PPDDISTRIBUTION = [
[0.2261905, 0.2857143, 0.2027027, 0.3817272, 0.2078767, 0.3080275, 0.4244343, 0.1842816, 0.3720966, 0.4858804, 0.5218593,
0.5293737, 0.3721124, 0.4493249, 0.6381578, 0.5146791, 0.5922679, 0.5332589, 0.5117431, 0.5629579, 0.5749365, 0.5019999],
[0.2852590, 0.2662420, 0.3536201, 0.3855332, 0.4075850, 0.4155194, 0.4510925, 0.4598753, 0.5888055, 0.5842944, 0.6055101, 
0.5802591, 0.6794434, 0.6630481, 0.6482930, 0.7274129, 0.7117230, 0.6689520, 0.6923058, 0.6233929, 0.5738572, 0.5687370],
[0.1293500, 0.2212608, 0.1848688, 0.1642599, 0.2324451, 0.2166240, 0.2226831, 0.2308148, 0.3169514, 0.4353170, 0.4403737, 
0.5057345, 0.4480123, 0.4210563, 0.4365776, 0.5464440, 0.4756535, 0.4334243, 0.5592600, 0.6745570, 0.5346794, 0.4550637],
[0.2672218, 0.3206308, 0.1667942, 0.2210191, 0.2513483, 0.2507335, 0.2919670, 0.3430441, 0.4016441, 0.4868953, 0.4584283, 
0.5431098, 0.4950821, 0.4924734, 0.7182794, 0.6640556, 0.4691735, 0.4397106, 0.4118190, 0.3247637, 0.2165149, 0.4062459],
[0.2569009, 0.2782784, 0.3168253, 0.3282276, 0.3414351, 0.3983651, 0.3999525, 0.4294075, 0.5298476, 0.5816537, 0.6182097, 
0.5764422, 0.6190777, 0.6112993, 0.6056073, 0.6402891, 0.6245472, 0.6202418, 0.6183364, 0.6055132, 0.5652483, 0.5399299],
[0.2023615, 0.2029750, 0.2238223, 0.2269491, 0.2405683, 0.2478660, 0.2792186, 0.3147041, 0.4197736, 0.4514789, 0.4393394, 
0.4704613, 0.4943117, 0.4691044, 0.5651123, 0.5572553, 0.5812581, 0.4830601, 0.5842594, 0.5639548, 0.5099648, 0.4978383],
[0.1324745, 0.1329507, 0.1284098, 0.1126005, 0.1521349, 0.1678299, 0.1559256, 0.2097362, 0.2198340, 0.3339065, 0.3383134, 
0.3091074, 0.3728780, 0.3972242, 0.4323098, 0.4819185, 0.4767688, 0.4320474, 0.4261401, 0.4030975, 0.4122228, 0.3806762],
[0.0913349, 0.1129764, 0.1030608, 0.1553628, 0.1806764, 0.1879892, 0.1861431, 0.2178382, 0.2463134, 0.3133631, 0.3384066, 
0.3713075, 0.4223481, 0.4015160, 0.3930278, 0.4229240, 0.3964960, 0.3686094, 0.4263992, 0.4670796, 0.3997695, 0.3845350],
[0.2486264, 0.4194631, 0.3417533, 0.3603083, 0.2315789, 0.3566465, 0.2206676, 0.3615312, 0.5033089, 0.4615174, 0.5123493, 
0.6075004, 0.6454838, 0.7500589, 0.6393598, 0.5575300, 0.6021262, 0.5947722, 0.5161463, 0.5989467, 0.4467263, 0.5667175]
]

Global functions:

In [7]:
def map_to_index(pp, j, o):
    '''
    Return state index
    
    :param int pp: 1 = receiving PP, 0 = not receiving PP
    :param int j: current job type
    :param bool o: True = changed job, False = kept job
    '''
    if o:
        _o = 1
    else:
        _o = 0            
    return pp*( len(o_range)*JOBS ) + j*len(o_range) + _o

def wage(j, pp, h, h_cum, o, z_shock):
    '''
    Earnings function
    
    :param int j: current job type
    :param int pp: 1 = receiving PP, 0 = not receiving PP
    :param float h: current hours worked
    :param float h_cum: cumulative hours worked
    :param bool o: True = changed job, False = kept job
    :param float z_shock: shock
    '''
    if h == 0:
        if h_cum < HMAX:
            return 0
        else:
            _log_h = 0
    else:
        _log_h = np.log(h)
    if o:
        _o = 1
    else:
        _o = 0
    return math.exp(
                WAGECONSTANT + 
                ALPHA*pp + 
                ZETA[j] + 
                GAMMA[0]*_log_h +
                GAMMA[1]*math.log(h_cum) + 
                GAMMA[2]*math.log(h_cum)*pp + 
                XI*_o +
                z_shock
                )

def consumption(a_prime, a, w):
    '''
    Consumption
    
    :param float a_prime: next period assets
    :param float a: current peruod assets
    :param float w: wage
    '''
    return (1 - TAU)*w + (1 + R)*a - a_prime
                
def labor(h_prime, h):
    '''
    Labor supply in hours
    
    :param float h_prime: next period cumulative hours worked
    :param float h: current peruod cumulative hours worked
    '''
    return h_prime - h
    
def utility(c, l):
    '''
    Utility function

    :param float c: consumption
    :param float l: labor supply
    '''
    return CHI*(c**(1-IOTA))/(1-IOTA) - (1 - CHI)*(l**(1+PSI))/(1+PSI) 
#     return (c**(-0.5) - 1)/(-0.5)

def augm(a):
    x = []
    for i in range(1, len(a)):
        x.append(a[i] - a[i-1])
    return np.array(x)

def save_to_csv(m, fn):
    _size = len(pp_range)*len(o_range)*JOBS*NBA*NBH
    _df = pd.DataFrame(np.array(m.dr).reshape(_size, ZSHOCKS),
                                columns=z_shock_range)
    _df.to_csv("dr-" + str(fn) + ".csv")
    _df = pd.DataFrame(np.array(m.v).reshape(_size, ZSHOCKS),
                                columns=z_shock_range)
    _df.to_csv("v-" + str(fn) + ".csv")

Model class:

In [8]:
class Model:
    '''
    Model
    '''

    def __init__(self, a_min=AMIN, a_max=AMAX, nb_a=NBA, augm_a=AUGMA,
                    h_min=HMIN, h_max=HMAX, nb_h=NBH, augm_h=AUGMH):
        '''
        :param float a_min: Minimal value of assets
        :param float a_max: Maximal value of assets
        :param int nb_a: Number of assets points in the grid
        :param float augm_a: Borrowing limit 
        :param float h_min: Minimal value of hours worked
        :param float h_max: Maximal value of hours worked
        :param int nb_h: Number of hours worked points in the grid
        :param float augm_h: Maximum hours worked per year

        '''
        (self.AMIN, self.AMAX, self.NBA) = (a_min, a_max, nb_a)
        (self.HMIN, self.HMAX, self.NBH) = (h_min, h_max, nb_h)
        self.AUGMA = augm_a
        self.AUGMH = augm_h
                
        if self.NBA > 1:
            self.DELTAA = (self.AMAX - self.AMIN) / (self.NBA - 1)     # Increment of assets
        else:
            self.DELTAA = 1
        if self.NBH > 1:
            self.DELTAH = (self.HMAX - self.HMIN) / (self.NBH - 1)     # Increment of hours worked
        else:
            self.DELTAH = 1
        
        self.a_grid = np.linspace(self.AMIN, self.AMAX, self.NBA)
        self.h_grid = np.linspace(self.HMIN, self.HMAX, self.NBH)
        
        self.util = []
        dr = np.ones( (self.NBA*self.NBH, ZSHOCKS), dtype=int )   #!!!!!
        v = np.zeros( (self.NBA*self.NBH, ZSHOCKS), dtype=float )
        self.v = [v] * ( len(pp_range)*len(o_range)*JOBS )
        self.dr = [dr] * ( len(pp_range)*len(o_range)*JOBS )
        

    def wage(self, j, pp, h_prime, h, o, z_shock):
        '''
        Earnings function on grid
        
        :param int j: current job type
        :param int pp: 1 = receiving PP, 0 = not receiving PP
        :param int h_prime: index of the next period cumulative hours worked
        :param int h: index of the current peruod cumulative hours worked
        :param bool o: True = changed job, False = kept job
        :param int z_shock: z-shock index
        '''
        if 2*h - h_prime < 0:
            _h_old = self.HMIN
        else:
            _h_old = self.HMIN + self.DELTAH*(2*h - h_prime)
        if h == self.NBH:
            _h = self.AUGMH*0.7
        else:
            _h = self.DELTAH*(h_prime - h)
        return self.wage( j, pp, _h, _h_old, o, z_shock_range[z_shock] )
    
    def consumption(self, j, pp, h_prime, h, a_prime, a, o, z_shock):
        '''
        Consumption on grid
    
        :param int j: current job type
        :param int pp: 1 = receiving PP, 0 = not receiving PP
        :param int h_prime: index of the next period cumulative hours worked
        :param int h: index of the current peruod cumulative hours worked
        :param float a_prime: index of the next period assets
        :param float a: index of the current peruod assets
        :param bool o: True = changed job, False = kept job
        :param int z_shock: z-shock index
        '''
#         _a_prime = self.a_grid[a_prime]
#         _a = self.a_grid[a]
#         return _a**0.3 + _a*0.9 - _a_prime
        _w = self.wage(j, pp, h_prime, h, o, z_shock)
#         return _w*(1 - TAU)
        return consumption( self.AMIN + self.DELTAA*a_prime, self.AMIN + self.DELTAA*a, _w )
                
    def labor(self, h_prime, h):
        '''
        Labor supply on grid
    
        :param int h_prime: index of the next period cumulative hours worked
        :param int h: index of the current peruod cumulative hours worked
        '''
        return self.DELTAH * labor(h_prime, h)  / self.AUGMH
        
    def utility(self, c, l):
        '''
        Utility function on grid

        :param float c: consumption on grid
        :param float l: labor supply on grid
        '''
        return CHI*(c**(1-IOTA))/(1-IOTA) - \
             (1 - CHI)*(l**(1+PSI))/(1+PSI)       #utility(c, l)
#         return (c**(-0.5) - 1)/(-0.5)
        
    def evaluate_model(self):
        if len(self.util) == 0:
            self.update_utility_matrix()
        self.iterate_model()
        
    def update_utility_matrix(self):
        print("Updating utility matrix", end='')
        for pp in pp_range:
            for j in range(JOBS):
                for o in o_range:
                    print(".", end='')
                    _temp_um = np.zeros((self.NBA*self.NBH, self.NBA*self.NBH, ZSHOCKS), dtype=float)
                    _temp_um.fill(np.nan) 
                
                    for z_shock in range(ZSHOCKS): 
                        for h_start in range(self.NBH):
                            for a_start in range(self.NBA):
                                for h_end in range(h_start, self.NBH): 
                                    if h_end - h_start > self.AUGMH/self.DELTAH:
                                        continue
                                    for a_end in range(self.NBA): ###!!!!!!!!!!!!!!!! not from a_start!
                                        if a_end - a_start > self.AUGMA/self.DELTAA:
                                            continue
                                        _c = self.consumption(j, pp, 
                                                            h_end, h_start, 
                                                            a_end, a_start,
                                                            o, z_shock);                                        

                                        if _c < 0:
                                            continue
                                        _l = self.labor(h_end, h_start)
                                        
#                                         print(a_start + h_start*self.NBA, 
#                                                     a_end + h_end*self.NBA)
                                        
                                        _temp_um[a_start + h_start*self.NBA, 
                                                    a_end + h_end*self.NBA, 
                                                    z_shock] = \
                                            self.utility(_c, _l)
                    self.util.append(_temp_um)
        print(" Done")

    def iterate_model(self):
        _temp_u0 = np.zeros( (self.NBA*self.NBH*ZSHOCKS), dtype=float )
        _distance = [1] * ( len(pp_range)*len(o_range)*JOBS )
        _iteration = 0
        
        while sum(_distance) > CRITERION and _iteration < MAXITERATIONS:
            _dr = []
            _temp_v = []

            for pp in pp_range:
                for j in range(JOBS):
                    for o in o_range:
                        if _distance[map_to_index(pp, j, o)] == 0:
                            _dr.append( self.dr[map_to_index(pp, j, o)] )
                            _temp_v.append( self.v[map_to_index(pp, j, o)] )
                            continue
                        
                        _dr.append( np.nanargmax(
                                    self.util[map_to_index(pp, j, o)] + 
                                    BETA*np.tile( self.v[map_to_index(pp, j, o)]
                                    .dot(PIMATRIX), 
                                    (self.NBA*self.NBH, 1, 1)), axis=1 ) )
                        
                        _Q = sp.lil_matrix( 
                                    (self.NBA*self.NBH*ZSHOCKS, self.NBA*self.NBH*ZSHOCKS), 
                                    dtype=float )

                        for z_shock in range(ZSHOCKS):
                            _Q0 = sp.lil_matrix( (self.NBA*self.NBH, self.NBA*self.NBH), 
                                                    dtype=float )
                            for i in range(self.NBA*self.NBH):
                                _Q0[i, _dr[map_to_index(pp, j, o)][i, 
                                                                z_shock]] = 1
                                _temp_u0[i + z_shock*self.NBA*self.NBH] = \
                                    self.util[map_to_index(pp, j, o)][i, 
                                        _dr[map_to_index(pp, j, o)][i, z_shock], 
                                        z_shock]
                            _Q[z_shock*self.NBA*self.NBH : (z_shock + 1)*self.NBA*self.NBH, :] = \
                                    sp.kron( PIMATRIX[:, z_shock], _Q0 )            

#                         _temp_u0 += np.euler_gamma
                        _solution = spsolve( 
                                    (sp.eye(self.NBA*self.NBH*ZSHOCKS) - BETA*_Q), 
                                    _temp_u0 )
                        _temp_v.append( 
                                    _solution.reshape( (ZSHOCKS, self.NBA*self.NBH)).T )

                        _distance[map_to_index(pp, j, o)] = np.max( abs( self.dr[map_to_index(pp, j, o)] -
                                                                         _dr[map_to_index(pp, j, o)] ) )
#             _distance = np.max( abs( np.array(self.v) - np.array(_temp_v) ) )

            _iteration +=1
            self.v = _temp_v
            self.dr = _dr

            print("Iteration: {0:2d}; Distance: {1:d}".format(_iteration, sum(_distance)))
            if _iteration == MAXITERATIONS:
                print("Max number of iterations reached!")
    
    def load_from_csv(self, fn):
        _df = pd.read_csv("dr-" + str(fn) + ".csv", index_col=0)
        self.dr = _df.as_matrix().reshape(len(pp_range)*len(o_range)*JOBS, 
                            NBA*NBH, ZSHOCKS)
        _df = pd.read_csv("v-" + str(fn) + ".csv", index_col=0)
        self.v = _df.as_matrix().reshape(len(pp_range)*len(o_range)*JOBS, 
                            NBA*NBH, ZSHOCKS)

        
    def show(self, i, j, k):
        plt.figure()
        plt.plot( self.h_grid, self.h_grid )
        plt.plot( self.h_grid,
                 self.h_grid[self.dr[i].
                 reshape(self.NBH, self.NBA, ZSHOCKS)[:, j, k] // self.NBA] )
        
        plt.figure()
        plt.plot( self.a_grid, self.a_grid )
        plt.plot( self.a_grid,
                 self.a_grid[self.dr[i].
                 reshape(self.NBH, self.NBA, ZSHOCKS)[j, :, k] % self.NBA] )

        plt.show()

Model evaluation:

In [ ]:
m = Model()

m.evaluate_model()

# m.load_from_csv(1)

In [ ]:
%matplotlib inline

m.show(map_to_index(1,0,False),0,2)
m.show(map_to_index(1,0,False),0,1)

In [ ]:
plt.plot(m.h_grid, m.v[0].reshape(m.NBH, m.NBA, ZSHOCKS)[:,-1, 0] )
plt.ylabel("value function")
plt.xlabel("cumulative hours worked")
plt.show()

plt.plot(m.a_grid, m.v[0].reshape(m.NBH, m.NBA, ZSHOCKS)[10,:, 0] )
plt.ylabel("value function")
plt.xlabel("assets")
plt.show()

In [ ]:
save_to_csv(m, 0)

In [ ]:
m.wage(0,0,1,0,False,0)

wage(0,0,1*m.DELTAH,m.HMIN,False,0), wage(0,0,1*m.DELTAH,m.HMIN,False,0.1)

Simulation utilities:

In [ ]:
def simulate_agent(m, log=True):
    _dr_idx = m.NBA // 1
    _h = m.HMIN + (_dr_idx // m.NBA)*m.DELTAH
    _a = m.AMIN + (_dr_idx % m.NBA)*m.DELTAA
    res = np.array( [(0, 0, 0, 0.0, 0.0, 0.0, 0.0)]*PERIODS )
    if log:
        print(" Period| Job| PP| Shock|     h|     a|    l|     c|     w ")
        print(" {0:6d}|   -|  -|     -|{1:6.0f}|{2: 6.0f}|    -|     -|     -".
                format(0, _h, _a) )
    _chosen = -1
    for period in range(PERIODS):            
        _temp_v = [0]*JOBS
        _temp_idx = [0]*JOBS
        _temp_state = [(0,0,0)]*JOBS
        for j in range(JOBS):
            _pp_prob = rnd.random()
            if (_pp_prob < PPDDISTRIBUTION[j][period]):
                pp = 1
            else:
                pp = 0
            _z_shock_prob = rnd.random()
            _z_index = 0
            _sum = ZDISTRIBUTION[_z_index]
            while (_sum < _z_shock_prob):
                _z_index += 1
                _sum += ZDISTRIBUTION[_z_index]
            _temp_idx[j] = m.dr[map_to_index(pp, j, 
                j != _chosen)].reshape(m.NBA*m.NBH, ZSHOCKS)[_dr_idx, _z_index]
            _temp_v[j] = m.v[map_to_index(pp, j, 
                j != _chosen)].reshape(m.NBA*m.NBH, ZSHOCKS)[_temp_idx[j], _z_index]
            _temp_state[j] = (pp, j == _chosen, z_shock_range[_z_index])
        _chosen = np.argmax(_temp_v)
        _l = labor(m.HMIN + (_temp_idx[_chosen] // m.NBA)*m.DELTAH, 
                    m.HMIN + (_dr_idx // m.NBA)*m.DELTAH) / m.AUGMH
        _w = wage( _chosen, 
                    _temp_state[_chosen][0], 
                    _l*m.AUGMH, 
                    m.HMIN + (_dr_idx // m.NBA)*m.DELTAH, 
                    _temp_state[_chosen][1],
                    _temp_state[_chosen][2] )
        
        _c = consumption(m.AMIN + (_temp_idx[_chosen] % m.NBA)*m.DELTAA,
                m.AMIN + (_dr_idx % m.NBA)*m.DELTAA, _w)
        _dr_idx = _temp_idx[_chosen]
        _h = m.HMIN + (_dr_idx // m.NBA)*m.DELTAH
        _a = m.AMIN + (_dr_idx % m.NBA)*m.DELTAA
        
        res[period] = (period + 1, _chosen, 
                    _temp_state[_chosen][0], 
                    _h, _l*AUGMH, _c, _w)
        if log:
#             print(_w, _chosen, 
#                     _temp_state[_chosen][0], _l,
#                     _l*m.AUGMH, 
#                     m.HMIN + (_dr_idx // m.NBA)*m.DELTAH, 
#                     _temp_state[_chosen][1],
#                     _temp_state[_chosen][2] )
            print(" {0:6d}|{1: 4d}|{2: 3d}|{3: 6.2f}|".
                    format(period + 1,
                    _chosen, 
                    _temp_state[_chosen][0], 
                    _temp_state[_chosen][2]) +
                    "{0:6.0f}|{1: 6.0f}|{2: 5.2f}|{3: 6.0f}|{4: 6.0f}".
                        format(_h, _a, _l, _c, _w) )
    return res
        
def integrate(m):
    average = np.array( [(0, 0, 0, 0.0, 0.0, 0.0, 0.0)]*PERIODS )
    for agent in range(AGENTS):
        average += simulate_agent(m, log=False)
    average /= AGENTS
#     print("     h|     a|    l|       c|       w ")
#     for period in range(PERIODS):   
#         (_h, _a, _l, _c, _w) = tuple(average[period])       
#         print("{0:6.0f}|{1: 6.0f}|{2: 5.2f}|{3: 8.0f}|{4: 8.2f}".
#                         format( _h, _a, _l, _c, _w ) )
    return average

Simulate one agent:

In [ ]:
res_1 = simulate_agent(m)

Simulate all agents:

In [ ]:
res = integrate(m)

Average consumption versus period:

In [ ]:
plt.figure()
plt.plot(np.linspace(1,PERIODS,PERIODS), np.log(res[:,6]))
plt.xlabel("Period")
plt.ylabel("Log earnings")
plt.figure()
plt.plot(np.linspace(1,PERIODS,PERIODS), np.log(res[:,3]))
plt.xlabel("Period")
plt.ylabel("Log cumulative hours worked")
plt.show()

In [ ]:
ar1 = res[1:,4]
ar2 = augm(res[:,0])

PSI = np.cov(ar1, ar2)[0,1] / (np.std(ar1) * np.std(ar2))
CHI = np.mean(res[:,2])

print("PSI: {0: 0.3f}\nCHI: {1: 0.3f}".format(PSI, CHI))

In [ ]:
PSI

In [ ]:
iser=pd.Series([1], dtype='int')
fser=pd.Series([1], dtype='float')

df = pd.DataFrame({'Agent':iser,'Period':iser,'Job':iser,'PP':iser,'h_cum':fser,'h':fser,'c':fser,'w':fser})


In [ ]:
df1=pd.DataFrame(res_1[:,0:3], dtype='int',columns=['Period','Job','PP'])
df2=pd.DataFrame(res_1[:,3:], dtype='float',columns=['h_cum','h','c','w'])
df.append(pd.concat([df1,df2], axis=1))

In [ ]:
res_1[:,0:3]